In [1]:
import numpy as np
import pandas as pd
import pickle
from itertools import product
import gc

In [2]:
x_train_list = pickle.load(open('../gen_data/x_train--simple_validation_split.ipynb--.pickle','rb'))
y_train_list = pickle.load(open('../gen_data/y_train--simple_validation_split.ipynb--.pickle','rb'))
y_val_list = pickle.load(open('../gen_data/y_val--simple_validation_split.ipynb--.pickle','rb'))
x_val_list = pickle.load(open('../gen_data/test_data_enriched--enrich1.ipynb--.pickle','rb'))

In [3]:
x_train = x_train_list[1]
y_train = y_train_list[1]
y_val = y_val_list[1]
x_val = x_val_list[1]

It can be seen that this data is missing a lot of 0 item sales

In [4]:
np.mean(y_train==0)

0.002173853599846873

In [5]:
train = x_train.copy()
train['item_cnt_month'] = y_train.copy()

# Create Target Encodings

In [6]:
item_sales = train.groupby(['date_block_num','item_id'],as_index=False).item_cnt_month.sum()
item_sales.columns = ['date_block_num','item_id','sum_item_sales_back_0']
train = train.merge(item_sales,how='left')
#put this in x_val
#then use OOF or permutations to get data for x_train to avoid overfitting
    #maybe just do the basic stuff for x_train, worry about overfitting later

In [7]:
shop_sales = train.groupby(['date_block_num','shop_id'],as_index=False).item_cnt_month.sum()
shop_sales.columns = ['date_block_num','shop_id','sum_shop_sales_back_0']
train = train.merge(shop_sales,how='left')

In [8]:
shop_item_sales = train.groupby(['date_block_num','shop_id','item_id'],as_index=False).agg({'item_cnt_month':'sum'})
shop_item_sales.columns = ['date_block_num','shop_id','item_id','item_cnt_month_back_0']
train = train.merge(shop_item_sales,how='left',on=['date_block_num','shop_id','item_id'])

In [9]:
train.head()

,date_block_num,shop_id,item_id,item_category_id,shop_name,item_name,item_category_name,item_cnt_month,sum_item_sales_back_0,sum_shop_sales_back_0,item_cnt_month_back_0
0,0,0,32,40,"!Якутск Орджоникидзе, 56 фран",1+1,Кино - DVD,6,242,5411,6
1,0,0,33,37,"!Якутск Орджоникидзе, 56 фран",1+1 (BD),Кино - Blu-Ray,3,61,5411,3
2,0,0,35,40,"!Якутск Орджоникидзе, 56 фран",10 ЛЕТ СПУСТЯ,Кино - DVD,1,78,5411,1
3,0,0,43,40,"!Якутск Орджоникидзе, 56 фран",100 МИЛЛИОНОВ ЕВРО,Кино - DVD,1,2,5411,1
4,0,0,51,57,"!Якутск Орджоникидзе, 56 фран",100 лучших произведений классики (mp3-CD) (Dig...,Музыка - MP3,2,13,5411,2


# Generate 0 Entries


Need to include entries where 0 sales were made for item/shop pairs in a month.
So this doesnt get out of hand, gonna focus only on all possible item/shop pairs based on sales in that month, 
this is what the coursera course did, see outside/Programming_assignment_week_4.ipynb for more info

In [10]:
months = range(train.date_block_num.min(),train.date_block_num.max()+1)
to_pandas=[]
print('Computing for month:',end=' ')
for month in months:
    print(month,end=', ')
    subtrain = train[train.date_block_num==month].copy()
    all_shops = subtrain.shop_id.unique()
    all_items = subtrain.item_id.unique()

    pairs = product(all_shops,all_items)
    to_pandas.append([(month,x[0],x[1]) for x in pairs])
    
train_filled = pd.DataFrame(np.vstack(to_pandas),columns=['date_block_num','shop_id','item_id'])
#train_filled['shop_id'] = train_filled.shop_id.astype(np.int32)
#train_filled['item_id'] = train_filled.item_id.astype(np.int32)


Computing for month: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 

In [11]:
train_filled = train_filled.merge(train,how='left',on=['date_block_num','shop_id','item_id'])
#entries where item_cnt_month is supposed to be 0 are now created as NA
train_filled.fillna(0,inplace=True)

In [12]:
train_filled.head()

,date_block_num,shop_id,item_id,item_category_id,shop_name,item_name,item_category_name,item_cnt_month,sum_item_sales_back_0,sum_shop_sales_back_0,item_cnt_month_back_0
0,0,0,32,40.0,"!Якутск Орджоникидзе, 56 фран",1+1,Кино - DVD,6.0,242.0,5411.0,6.0
1,0,0,33,37.0,"!Якутск Орджоникидзе, 56 фран",1+1 (BD),Кино - Blu-Ray,3.0,61.0,5411.0,3.0
2,0,0,35,40.0,"!Якутск Орджоникидзе, 56 фран",10 ЛЕТ СПУСТЯ,Кино - DVD,1.0,78.0,5411.0,1.0
3,0,0,43,40.0,"!Якутск Орджоникидзе, 56 фран",100 МИЛЛИОНОВ ЕВРО,Кино - DVD,1.0,2.0,5411.0,1.0
4,0,0,51,57.0,"!Якутск Орджоникидзе, 56 фран",100 лучших произведений классики (mp3-CD) (Dig...,Музыка - MP3,2.0,13.0,5411.0,2.0


# Create Lag Values

In [13]:
#function to downcast data types to 32 bits
def downcast(df):
    float_cols = [col for col in df if df[col].dtype=='float64']
    int_cols = [col for col in df if df[col].dtype=='int64']

    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int32)
    
    return(df)

In [14]:
#gonna iterively copy a subset of the data, rename the date block and aome other cols then merge it back in dateback_gen = range(1,13)
dateback_gen = range(1,13)
lag_train_filled = downcast(train_filled)
del train_filled
del train
print('dateback=',end=' ')
gc.collect()
for dateback in dateback_gen: 
    #this line inneficient?
    to_shift = lag_train_filled[['date_block_num','shop_id','item_id','sum_item_sales_back_0','sum_shop_sales_back_0','item_cnt_month_back_0']].copy()
    to_shift['date_block_num'] = to_shift.date_block_num + dateback
    newcols = ['date_block_num','shop_id','item_id','sum_item_sales_back_'+str(dateback),'sum_shop_sales_back_'+str(dateback),'item_cnt_month_back_'+str(dateback)]
    to_shift.columns = newcols
    #print(newcols)
    print(dateback,end=', ')
    lag_train_filled = lag_train_filled.merge(to_shift,on=['date_block_num','shop_id','item_id'],how='left').fillna(0)
    del to_shift
    gc.collect()


dateback= 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 

In [15]:
lag_train_filled[lag_train_filled.date_block_num>=12].shape
#This should have 6425094 to mimmick the course script (except this is using all of the shops)

(6425094, 47)

In [16]:
lag_train_filled.head()

,date_block_num,shop_id,item_id,item_category_id,shop_name,item_name,item_category_name,item_cnt_month,sum_item_sales_back_0,sum_shop_sales_back_0,...,item_cnt_month_back_9,sum_item_sales_back_10,sum_shop_sales_back_10,item_cnt_month_back_10,sum_item_sales_back_11,sum_shop_sales_back_11,item_cnt_month_back_11,sum_item_sales_back_12,sum_shop_sales_back_12,item_cnt_month_back_12
0,0,0,32,40.0,"!Якутск Орджоникидзе, 56 фран",1+1,Кино - DVD,6.0,242.0,5411.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,33,37.0,"!Якутск Орджоникидзе, 56 фран",1+1 (BD),Кино - Blu-Ray,3.0,61.0,5411.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,35,40.0,"!Якутск Орджоникидзе, 56 фран",10 ЛЕТ СПУСТЯ,Кино - DVD,1.0,78.0,5411.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,43,40.0,"!Якутск Орджоникидзе, 56 фран",100 МИЛЛИОНОВ ЕВРО,Кино - DVD,1.0,2.0,5411.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,51,57.0,"!Якутск Орджоникидзе, 56 фран",100 лучших произведений классики (mp3-CD) (Dig...,Музыка - MP3,2.0,13.0,5411.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Checks

In [17]:
lag_train_filled[(lag_train_filled.shop_id == 28) & (lag_train_filled.item_id==10991)&(lag_train_filled.date_block_num==12)].sum_shop_sales_back_4

4648046    0.0
Name: sum_shop_sales_back_4, dtype: float32

In [18]:
lag_train_filled[(lag_train_filled.shop_id == 28) & (lag_train_filled.item_id==10991)&(lag_train_filled.date_block_num==8)].sum_shop_sales_back_0

3178044    0.0
Name: sum_shop_sales_back_0, dtype: float32

In [19]:
x = lag_train_filled

In [20]:
x[(x.date_block_num==32) & (x.shop_id==18) & (x.item_id==32)].sum_shop_sales_back_4

10514168    1170.0
Name: sum_shop_sales_back_4, dtype: float32

In [21]:
x[(x.date_block_num==28) & (x.shop_id==18) & (x.item_id==32)].sum_shop_sales_back_0

9615682    1170.0
Name: sum_shop_sales_back_0, dtype: float32

In [22]:
x[(x.date_block_num==26) & (x.shop_id==10) & (x.item_id==33)].sum_shop_sales_back_0

9057267    541.0
Name: sum_shop_sales_back_0, dtype: float32

In [23]:
x[(x.date_block_num==27) & (x.shop_id==10) & (x.item_id==33)].sum_shop_sales_back_1

9337818    541.0
Name: sum_shop_sales_back_1, dtype: float32

# Remove first 12 months
This data doesnt have correct history

In [24]:
train = lag_train_filled[lag_train_filled.date_block_num>=12]
del lag_train_filled

In [25]:
train.head()

,date_block_num,shop_id,item_id,item_category_id,shop_name,item_name,item_category_name,item_cnt_month,sum_item_sales_back_0,sum_shop_sales_back_0,...,item_cnt_month_back_9,sum_item_sales_back_10,sum_shop_sales_back_10,item_cnt_month_back_10,sum_item_sales_back_11,sum_shop_sales_back_11,item_cnt_month_back_11,sum_item_sales_back_12,sum_shop_sales_back_12,item_cnt_month_back_12
4488756,12,2,32,40.0,"Адыгея ТЦ ""Мега""",1+1,Кино - DVD,1.0,84.0,863.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4488757,12,2,33,37.0,"Адыгея ТЦ ""Мега""",1+1 (BD),Кино - Blu-Ray,1.0,42.0,863.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,1146.0,1.0
4488758,12,2,99,37.0,"Адыгея ТЦ ""Мега""",1812: УЛАНСКАЯ БАЛЛАДА (BD),Кино - Blu-Ray,1.0,26.0,863.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4488759,12,2,482,73.0,"Адыгея ТЦ ""Мега""",1С:Бухгалтерия 8. Базовая версия,Программы - 1С:Предприятие 8,2.0,111.0,863.0,...,1.0,128.0,751.0,1.0,95.0,489.0,1.0,118.0,1146.0,1.0
4488760,12,2,485,73.0,"Адыгея ТЦ ""Мега""",1С:Бухгалтерия 8. Учебная версия. Издание 7.,Программы - 1С:Предприятие 8,1.0,126.0,863.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Regularise Training Target encoding for
# --//--\_back\_0

In [26]:
sum_item_sales_back_0 = train.sum_item_sales_back_0
sum_shop_sales_back_0 = train.sum_shop_sales_back_0
item_cnt_month_back_0 = train.item_cnt_month_back_0

train = train.drop(['sum_item_sales_back_0','sum_shop_sales_back_0','item_cnt_month_back_0','item_cnt_month'],axis=1)

In [27]:
train

,date_block_num,shop_id,item_id,item_category_id,shop_name,item_name,item_category_name,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,...,item_cnt_month_back_9,sum_item_sales_back_10,sum_shop_sales_back_10,item_cnt_month_back_10,sum_item_sales_back_11,sum_shop_sales_back_11,item_cnt_month_back_11,sum_item_sales_back_12,sum_shop_sales_back_12,item_cnt_month_back_12
4488756,12,2,32,40.0,"Адыгея ТЦ ""Мега""",1+1,Кино - DVD,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4488757,12,2,33,37.0,"Адыгея ТЦ ""Мега""",1+1 (BD),Кино - Blu-Ray,42.0,1255.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,1146.0,1.0
4488758,12,2,99,37.0,"Адыгея ТЦ ""Мега""",1812: УЛАНСКАЯ БАЛЛАДА (BD),Кино - Blu-Ray,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4488759,12,2,482,73.0,"Адыгея ТЦ ""Мега""",1С:Бухгалтерия 8. Базовая версия,Программы - 1С:Предприятие 8,88.0,1255.0,1.0,...,1.0,128.0,751.0,1.0,95.0,489.0,1.0,118.0,1146.0,1.0
4488760,12,2,485,73.0,"Адыгея ТЦ ""Мега""",1С:Бухгалтерия 8. Учебная версия. Издание 7.,Программы - 1С:Предприятие 8,111.0,1255.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4488761,12,2,804,49.0,"Адыгея ТЦ ""Мега""",1С:Предприятие 8.2. Практическое пособие разра...,Книги - Методические материалы 1С,26.0,1255.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,1146.0,1.0
4488762,12,2,839,73.0,"Адыгея ТЦ ""Мега""",1С:Упрощенка 8,Программы - 1С:Предприятие 8,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,78.0,489.0,1.0,74.0,1146.0,1.0
4488763,12,2,1007,67.0,"Адыгея ТЦ ""Мега""",3D Crystal Puzzle Замок XL,Подарки - Развитие,247.0,1255.0,1.0,...,0.0,133.0,751.0,2.0,71.0,489.0,1.0,12.0,1146.0,3.0
4488764,12,2,1406,28.0,"Адыгея ТЦ ""Мега""",Aliens: Colonial Marines. Расширенное издание ...,Игры PC - Дополнительные издания,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4488765,12,2,1407,19.0,"Адыгея ТЦ ""Мега""",Aliens: Colonial Marines. Расширенное издание ...,Игры - PS3,466.0,1255.0,13.0,...,0.0,82.0,751.0,1.0,158.0,489.0,1.0,0.0,0.0,0.0


In [37]:
#tmp save
#pickle.dump(train,open('../gen_data/train.tmp.pickle','wb'))

In [38]:
#del train

In [28]:
#pickle.dump(open('../gen_data/features1','wb'))

In [29]:
test = pickle.load(open('../gen_data/test_data_enriched--enrich1.ipynb--.pickle','rb'))

In [30]:
test = test[1]

In [31]:
lag_test_filled = downcast(test)

In [32]:
lag_test_filled

,date_block_num,shop_id,item_id,item_category_id,shop_name,item_name,item_category_name
0,34,5,5037,19.0,"Вологда ТРЦ ""Мармелад""","NHL 15 [PS3, русские субтитры]",Игры - PS3
1,34,5,5320,NaN,NaN,NaN,NaN
2,34,5,5233,19.0,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Essentials) [PS3, русск...",Игры - PS3
3,34,5,5232,23.0,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Classics) [Xbox 360, ру...",Игры - XBOX 360
4,34,5,5268,NaN,NaN,NaN,NaN
5,34,5,5039,23.0,"Вологда ТРЦ ""Мармелад""","NHL 15 [Xbox 360, русские субтитры]",Игры - XBOX 360
6,34,5,5041,20.0,"Вологда ТРЦ ""Мармелад""","NHL 16 [PS4, русские субтитры]",Игры - PS4
7,34,5,5046,55.0,"Вологда ТРЦ ""Мармелад""",NICKELBACK No Fixed Address,Музыка - CD локального производства
8,34,5,5319,55.0,"Вологда ТРЦ ""Мармелад""",ONE DIRECTION Four,Музыка - CD локального производства
9,34,5,5003,NaN,NaN,NaN,NaN


In [33]:
ref_date_block = lag_test_filled.date_block_num.mean()
hist_data = train[train.date_block_num==ref_date_block-1]
  

In [34]:
test

,date_block_num,shop_id,item_id,item_category_id,shop_name,item_name,item_category_name
0,34,5,5037,19.0,"Вологда ТРЦ ""Мармелад""","NHL 15 [PS3, русские субтитры]",Игры - PS3
1,34,5,5320,NaN,NaN,NaN,NaN
2,34,5,5233,19.0,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Essentials) [PS3, русск...",Игры - PS3
3,34,5,5232,23.0,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Classics) [Xbox 360, ру...",Игры - XBOX 360
4,34,5,5268,NaN,NaN,NaN,NaN
5,34,5,5039,23.0,"Вологда ТРЦ ""Мармелад""","NHL 15 [Xbox 360, русские субтитры]",Игры - XBOX 360
6,34,5,5041,20.0,"Вологда ТРЦ ""Мармелад""","NHL 16 [PS4, русские субтитры]",Игры - PS4
7,34,5,5046,55.0,"Вологда ТРЦ ""Мармелад""",NICKELBACK No Fixed Address,Музыка - CD локального производства
8,34,5,5319,55.0,"Вологда ТРЦ ""Мармелад""",ONE DIRECTION Four,Музыка - CD локального производства
9,34,5,5003,NaN,NaN,NaN,NaN


In [35]:
test.merge(hist_data,on=['shop_id','item_id','item_category_id','shop_name','item_name','item_category_name'],how='left')

,date_block_num_x,shop_id,item_id,item_category_id,shop_name,item_name,item_category_name,date_block_num_y,sum_item_sales_back_1,sum_shop_sales_back_1,...,item_cnt_month_back_9,sum_item_sales_back_10,sum_shop_sales_back_10,item_cnt_month_back_10,sum_item_sales_back_11,sum_shop_sales_back_11,item_cnt_month_back_11,sum_item_sales_back_12,sum_shop_sales_back_12,item_cnt_month_back_12
0,34,5,5037,19.0,"Вологда ТРЦ ""Мармелад""","NHL 15 [PS3, русские субтитры]",Игры - PS3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34,5,5320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34,5,5233,19.0,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Essentials) [PS3, русск...",Игры - PS3,33.0,80.0,1047.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34,5,5232,23.0,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Classics) [Xbox 360, ру...",Игры - XBOX 360,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34,5,5268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,34,5,5039,23.0,"Вологда ТРЦ ""Мармелад""","NHL 15 [Xbox 360, русские субтитры]",Игры - XBOX 360,33.0,93.0,1047.0,...,1.0,189.0,1953.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
6,34,5,5041,20.0,"Вологда ТРЦ ""Мармелад""","NHL 16 [PS4, русские субтитры]",Игры - PS4,33.0,245.0,1047.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,34,5,5046,55.0,"Вологда ТРЦ ""Мармелад""",NICKELBACK No Fixed Address,Музыка - CD локального производства,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,34,5,5319,55.0,"Вологда ТРЦ ""Мармелад""",ONE DIRECTION Four,Музыка - CD локального производства,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,34,5,5003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# What if something in train didnt have a item cat id, (or another variable)? would the merge miss it?